# 01 — Constructing the Discrete Laplacian

In $\mathbb{R}^n$, the continuous setting, the Laplacian $\Delta$ is analytically defined as $$\Delta = \sum_{j=1}^{n} \frac{\partial^2}{\partial x_{j}^{2}}.$$

On a curved surface, the corresponding generalization is the *Laplace-Beltrami operator*. In practise, we represent surfaces as triangle meshes and discretize Laplacian as a  matrix. In this notebook, we will study the laplacian in this discrete setting. 

---
In the continuous setting, the laplacian acts on functions or surfaces. This raises the natural question: **What does the discrete laplacian act on?** To answer this we introduce some terminology. Assume the mesh has $V$ vertices. A scalar function on the mesh is just a vector
$$
u \in \mathbb{R}^V,
$$
where $u_i$ is the value of the function at vertex $i$.  A discrete Laplacian is a $V\times V$ matrix $L$ such that $(Lu)_i$ approximates “how $u$ bends” near vertex $i$. We now introduce two common ways to define the Laplacian on meshes.

---

## 1) Uniform graph Laplacian: 

This version only uses *connectivity* of the graph, not geometry. We first introduce two common matrices related to graphs.

The *adjacency matrix* $A$ associated to a graph $(V,E)$ is a $|V| \times |V|$ matrix defined as

$$
A_{ij} =
\begin{cases}
1 & \text{if vertices }\, i \, \text{ and } j \, \text{are adjacent} \\
0 & \text{otherwise.}
\end{cases}
$$

The *Degree matrix* $D$ is a diagonal matrix and is defined as

$$
D_{ii} = \sum_j A_{ij} = \text{degree of vertex i}, \qquad D_{ij}=0\ (i\neq j).
$$

Then uniform graph laplacian $L$ is defined as $L \coloneqq D - A$.

**Downside:** it treats every edge equally, even if some triangles are smaller or edges are long/short. So it doesnt provide a very accurate description of the $3$-d structure of the mesh.

---

## 2) Cotangent Laplacian 

This is the standard Laplace–Beltrami discretization for triangle meshes and takes into account the angles between edges. Thus in a sense its more geometry aware than the Unifrom graph laplacian.

For an edge $(i,j)$, let $\alpha_{ij}$ and $\beta_{ij}$ be the angles **opposite** that edge in the two incident triangles:

$$
L_{ij} = -\frac{1}{2}\big(\cot \alpha_{ij} + \cot \beta_{ij}\big)
\quad \text{if } i  \,\text{is adjacent to}\, j.
$$

- If $(i,j)$ is a **boundary edge**, there is only one incident triangle, so we only ave one angle and the other term is treated as $0$.

The diagonal is chosen so rows sum to zero:

$
L_{ii} = -\sum_{j\neq i} L_{ij}.
$

And if $i$ and $j$ are not adjacent then $L_{ij}=0$.

---

## Mass matrix $M$

The cotangent Laplacian $L$ is the discrete version of the operator $-\Delta$. But on a surface, we also need a way to discretize *integrals* like $\int_S f(x)\,dA$ and inner products like
$$
\langle f,g\rangle = \int_S f(x)\,g(x)\,dA.
$$

To do that, we introduce tha *mass matrix*. We define the mass matrix $M$ so that multiplying by it behaves like integrating over a surface:

- For a scalar function $u\in\mathbb{R}^V$ on the mesh, $Mu$ is like “weighted area averaging” the values of $u$.
- And the discrete inner product becomes
  $$
  \langle u,v\rangle_M := u^T M v \approx \int_S u(x)v(x)\,dA.
  $$

A very common simple choice is a **diagonal** (a.k.a. “lumped”) mass matrix where each vertex gets a share of nearby triangle area:
$$
M_{ii} = A_i,\qquad M_{ij}=0\ (i\neq j).
$$

With **barycentric area**,
$$
A_i = \frac{1}{3}\sum_{i \ni t}\text{Area}(t),
$$
where we sum over triangles that contain the vertex $i$. We scale by a factor of $1/3$ so as to split that triangle’s area equally among the three vertices incident to it.

In the continuous setting, Laplace–Beltrami eigenfunctions satisfy
$$
-\Delta \phi = \lambda \phi.
$$

After discretization, the operator part becomes $L$ but the “right-hand side” still represents multiplication by area (because of the inner product / integration). That turns the problem into a *generalized* eigenproblem:
$$
L\phi = \lambda M\phi.
$$

Intuitively: $L$ measures “how much the function bends”, while $M$ tells us how to measure size on the surface (it’s similar to the discrete version of $dA$).

In the next cells, we’ll assemble $L$ (cotangent weights) and $M$ (vertex areas) as matrices.


In [3]:
import os, sys

def _find_repo_root():
    for candidate in [os.getcwd(), os.path.abspath('..')]:
        if os.path.isdir(os.path.join(candidate, 'src')):
            return candidate
    raise FileNotFoundError("Can't find repo root. Run from the repo directory.")

ROOT = _find_repo_root()
sys.path.insert(0, ROOT)

import numpy as np
import matplotlib.pyplot as plt
from src.laplacian import (
    load_mesh, cotangent_weights, mass_matrix, uniform_laplacian
)

%matplotlib inline

## Generate sample meshes

In [17]:
# Generate meshes
import os
if not os.path.exists(f'{ROOT}/meshes/sphere.obj'):
    !cd {ROOT} && python src/generate_meshes.py

V_sphere, F_sphere = load_mesh(f'{ROOT}/meshes/sphere.obj')
V_torus, F_torus = load_mesh(f'{ROOT}/meshes/torus.obj')

print(f'Sphere: {len(V_sphere)} vertices, {len(F_sphere)} faces')
print(f'Torus:  {len(V_torus)} vertices, {len(F_torus)} faces')

Sphere: 642 vertices, 1280 faces
Torus:  1800 vertices, 3600 faces


## Build and inspect the Laplacians

In [7]:
L_cot = cotangent_weights(V_sphere, F_sphere)
L_uni = uniform_laplacian(V_sphere, F_sphere)
M = mass_matrix(V_sphere, F_sphere)

print(f'Cotangent Laplacian: shape={L_cot.shape}, nnz={L_cot.nnz}')
print(f'Uniform Laplacian:   shape={L_uni.shape}, nnz={L_uni.nnz}')
print(f'Mass matrix diagonal: min={M.diagonal().min():.6f}, max={M.diagonal().max():.6f}')
print(f'Total surface area:   {M.diagonal().sum():.4f}  (sphere should be ≈ 4π ≈ {4*np.pi:.4f})')

Cotangent Laplacian: shape=(642, 642), nnz=4482
Uniform Laplacian:   shape=(642, 642), nnz=4482
Mass matrix diagonal: min=0.015138, max=0.022682
Total surface area:   12.5065  (sphere should be ≈ 4π ≈ 12.5664)


## Sparsity 

The discrete Laplacian is *very sparse*. That means row $i$ of $L$ has nonzeros only in columns $j$ where $j=i$ or if they are adjacent. So each row typically has about $\text{deg}(i)+1$ nonzeros. This is illustrated in the cell below.

In [45]:
print("nnz(L_cot) =", L_cot.nnz, "density =", L_cot.nnz/(L_cot.shape[0]**2))
print("avg nnz/row =", (np.diff(L_cot.tocsr().indptr)).mean())

nnz(L_cot) = 4482 density = 0.010874312167001484
avg nnz/row = 6.981308411214953


## Verify properties

For a smooth function $\varphi$ (with appropriate boundary conditions), integration by parts gives

$$
\int \overline{\varphi(x)}\,\Delta \varphi(x)\,dx
\;=\;
-\int \|\nabla \varphi(x)\|^2\,dx
\;\le\;0.
$$
So the Laplacian is *negative semidefinite*.

In the discrete setting we show that it satisfies the following properties: 

- **Symmetry**: $L = L^T$. This guarantees real eigenvalues.
- **Row sums = 0**: $L \mathbf{1} = 0$. The analogous property in the continuous setting is that constant functions are in the null space.
- **Negative semi-definite**: For all $u \in \mathbb{R}^{|V|}$ there holds $u^T L u \le 0$. We verify this by showing all eigenvalues non positive.

In the cell below, we verify these properties.

In [37]:
# Symmetry check
sym_error = np.abs(L_cot - L_cot.T).max()
print(f'Symmetry error (max |L - L^T|): {sym_error:.2e}')

# Row sum check
row_sums = np.abs(L_cot @ np.ones(L_cot.shape[0]))
print(f'Row sum error (max |L·1|):      {row_sums.max():.2e}')

# Quick eigenvalue check (just a few)
from scipy.sparse.linalg import eigsh
eigs = eigsh(L_cot, k=6, which='SM', return_eigenvectors=False)
print(f'Smallest 6 eigenvalues:         {np.sort(eigs)}')
print('All ≤ 0' if np.all(eigs < 1e-8) else 'Issue detected')

Symmetry error (max |L - L^T|): 0.00e+00
Row sum error (max |L·1|):      7.77e-16
Smallest 6 eigenvalues:         [-1.16132968e-01 -1.16132968e-01 -3.89538294e-02 -3.89538294e-02
 -3.89538294e-02 -5.34543054e-16]
All ≤ 0


## Next

In the next notebook, we'll introduce the spectral decomposition and visualize the eigenvectors.

## References

- Meyer, M., Desbrun, M., Schröder, P., & Barr, A. H. *Discrete Differential-Geometry Operators for Triangulated 2-Manifolds* (2003).  
- Pinkall, U., & Polthier, K. *Computing Discrete Minimal Surfaces and Their Conjugates* (Experimental Mathematics, 1993).  
- Wardetzky, M., Mathur, S., Kälberer, F., & Grinspun, E. *Discrete Laplace Operators: No Free Lunch* (SGP, 2007).  